In [289]:
# import stuff
%matplotlib inline

from __future__ import print_function
from __future__ import division

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import statsmodels.api as sm

from statsmodels.tools import eval_measures
import statsmodels.formula.api as smf

from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from pyts.preprocessing import InterpolationImputer

# just for the sake of this blog post!
from warnings import filterwarnings
filterwarnings('ignore')

# load the provided data
train_features = pd.read_csv('data-processed/dengue_features_train.csv',
                             index_col=[0,1,2])

train_labels = pd.read_csv('data-processed/dengue_labels_train.csv',
                           index_col=[0,1,2])

# Seperate data for San Juan
sj_train_features = train_features.loc['sj']
sj_train_labels = train_labels.loc['sj']

# Separate data for Iquitos
iq_train_features = train_features.loc['iq']
iq_train_labels = train_labels.loc['iq']

In [290]:
# run all preprocessing steps on both cities, then in last step seperate them

# load data
X_path = 'data-processed/dengue_features_train.csv'
y_path = "data-processed/dengue_labels_train.csv"

def preprocess_data( X_path, labels):
    
    # Input: 
    # labels: logical true or false whether we have y (because submission data does not)
    
    # load data and set index to city, year, weekofyear and remove week_start_date
    df = pd.read_csv(X_path, index_col=[0, 1, 2])
    df = df.drop('week_start_date',axis=1)#[features]

    # fill missing values with interpolation
    df.fillna(method='bfill', inplace=True) # bfill: use next valid observation to fill gap

    ### add lag ###
    lag        = 3
    var2change = ['station_max_temp_c', 
                  'station_precip_mm',
                  'reanalysis_relative_humidity_percent',
                  'station_avg_temp_c']

    new_names = []
    for i,j in enumerate(var2change):
        new_names.append( j + '_lag')

        df[new_names[i]] = df[var2change[i]].shift(lag)  # Lagged by 1 time step
    
        # remove original 
        df = df.drop(var2change[i],axis=1)

    # remove missing values again because of lag
    df.fillna(method='bfill', inplace=True)

    
    ### create interaction feature: humidity above 42 % temperature above 26 degrees
    
    df['Humid_X_Temp26'] = np.where((df['reanalysis_relative_humidity_percent_lag'] >= 42) & (df['station_avg_temp_c_lag'] >= 26), 1, 0)    

    if labels==True:
        # add predictor to DF to seperate the cities
        y_path = "data-processed/dengue_labels_train.csv"
        y = pd.read_csv(y_path, index_col=[0, 1, 2])
        df = df.join(y)
    
    # separate san juan and iquitos
    df_sj = df.loc['sj']
    df_iq = df.loc['iq']

    if labels==True:
        # remove y from X again
        # San Juan
        X_sj = df_sj.drop('total_cases',axis=1)
        y_sj = df_sj['total_cases']

        # Iquitos
        X_iq = df_iq.drop('total_cases',axis=1)
        y_iq = df_iq['total_cases']
        
    else:
        X_sj = df_sj
        X_iq = df_iq
        y_sj = []
        y_iq = []
            
    return X_sj, y_sj, X_iq, y_iq

# remove df later again, just for debugging
[X_sj, y_sj, X_iq, y_iq] = preprocess_data( X_path, True)


In [291]:
# run OLS, check for non-important variables

X = sm.add_constant(X_sj, prepend=False)
y = y_sj

# Fit and summarize OLS model
mod = sm.OLS(y, X).fit()
#print(mod.summary())

In [292]:
# remove some unimportant regressors for San Juan accoding to OLS
#X_sj = X_sj.drop(columns=['ndvi_se','ndvi_sw'])

In [293]:
# Seperate X into prior and future train vs. test sets

# San Juan is 936 x 21
X_sj_train = X_sj.iloc[1:701]
X_sj_test = X_sj.iloc[701:]

# Iquitos is 520x20: split 390 vs 130
X_iq_train = X_iq.iloc[1:391]
X_iq_test = X_iq.iloc[391:]

In [294]:
# SJ
y_sj_train = y_sj.iloc[1:701]
y_sj_test  = y_sj.iloc[701:]

# IQ
y_iq_train = y_iq.iloc[1:391]
y_iq_test  = y_iq.iloc[391:]

In [295]:
# my version benchmarking function

# San Juan
# best alpha =  1e-08
# best score =  756.9779411764706

# Peru
# best alpha =  1e-08
# best score =  119.31666666666666

def get_score(X_train, y_train, X_test, y_test):
    
    rf_regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
    rf_regressor.fit(X_train, y_train)

    y_pred = rf_regressor.predict(X_test).astype(int)
    my_mse = metrics.mean_squared_error(y_test, y_pred)
    
    return my_mse, y_pred, rf_regressor

# output MSE 
[mse_sj, sj_pred, model_sj] = get_score(X_sj_train, y_sj_train, X_sj_test, y_sj_test)
print('San Juan: ' + str(mse_sj))

[mse_iq, iq_pred, model_iq] = get_score(X_iq_train, y_iq_train, X_iq_test, y_iq_test)
print('Iquitos: ' + str(mse_iq))


San Juan: 1153.9744680851063
Iquitos: 159.5503875968992


In [286]:
# baseline model with 4 features: 1802.2099802009457
# baseline model with all features: 1458.708840425532 

# experiment on interpolation
# after interpolation with 4 feature: 1906.9913617021275 
# after interpolation with all features 1762.8954468085103 with all features using ffill
# after interpolation with all features 11281.0836463829787 with all features using bfill

# experiment on lag, removing original avg_temp
# baseline model, 4 features, bfill: 1281.0836463829787
# baseline model, 4 features, bfill, lagged by 1 avg, temp: 1623.894638297872
# baseline model, 4 features, bfill, lagged by 2 avg, temp: 1454.7012340425533
# baseline model, 4 features, bfill, lagged by 3 avg, temp: 1361.6014042553193

# experiment on lag, keeping original avg_temp
# baseline model, 4 features, bfill: 1281.0836463829787
# baseline model, 4 features, bfill, lagged by 1 avg, temp:

# experiment on lag, removing original humidity
# 1 weeks: 1538.7760425531915
# 2 weeks: 1617.551914893617
# 3 weeks: 1429.1278297872343

# experiment on lag, keeping original humidity
# 2 weeks: 1617.551914893617
# 1458.708840425532
# 1524.5711259574466

# 3 week lag, max temp, removing original, bfill
# 1221.451829787234 

# max and precipitation
# 1185.6852765957449

# max, precip and humidity
# 1135.5494893617022

# removing SE and SW
#1128.7711063829786

# interaction
# San Juan: 1153.9744680851063
# Iquitos: 159.5503875968992

In [287]:
# list of changes to keep

### BETTER ###
# fill missing values
# df.fillna(method='bfill', inplace=True) better than ffill


### Worse ###
# avg_temp lag of 1-4 all worse

In [288]:
sj_test, sj_y, iq_test, iq_y = preprocess_data('data-processed/dengue_features_test.csv', False)

sj_predictions = model_sj.predict(sj_test).astype(int)
iq_predictions = model_iq.predict(iq_test).astype(int)

submission = pd.read_csv("data-processed/submission_format.csv",
                         index_col=[0, 1, 2])

submission.total_cases = np.concatenate([sj_predictions, iq_predictions])
submission.to_csv("data-processed/our_model1.csv")